In [579]:

import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
driver = webdriver.Chrome(ChromeDriverManager().install())
import pandas as pd
import re

#driver = webdriver.Chrome(executable_path='C:/Users/mdedhia/chromedriver')
#driver.get("https://www.google.com")


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\manan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [580]:
driver.get("https://www.airbnb.com/")

In [581]:
driver.get("https://www.airbnb.com/s/New-York--NY--United-States/homes?search_type=autocomplete_click&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJOwg_06VPwokRYv534QaPC8g&query=New%20York%2C%20NY%2C%20United%20States&source=structured_search_input_header")

In [582]:
all_pages = driver.find_element_by_class_name("_99vlue")
all_pages.text

'1\n2\n3\n4\n5\n…\n15\n1 – 20 of 300+ places to stay'

In [583]:
all_pages.text.split("\n")

['1', '2', '3', '4', '5', '…', '15', '1 – 20 of 300+ places to stay']

In [584]:
total_pages = int(all_pages.text.partition("…")[2].split("\n")[1])
total_pages

15

In [585]:
no_of_listings = int(all_pages.text.partition("…")[2].split(" ")[2])
no_of_listings

20

In [586]:
type(data)
temp =[]

In [587]:
data = pd.DataFrame(columns=['host_type', 'room_type', 'location', 'room_desc', 'no_of_guests','ratings','no_of_reviews','price'])
#data.set_axis(['host_type', 'room_type', 'location', 'room_desc', 'no_of_guests','ratings','no_of_reviews','price'], axis=1, inplace=False)
data

,host_type,room_type,location,room_desc,no_of_guests,ratings,no_of_reviews,price


In [588]:
for i in range(0,total_pages):
    offset_no = no_of_listings*i 
    driver.get(f"https://www.airbnb.com/s/New-York--NY--United-States/homes?search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&source=structured_search_input_header&gps_lat=37.489460400000006&gps_lng=-122.2284897&place_id=ChIJOwg_06VPwokRYv534QaPC8g&federated_search_session_id=3fb8e1de-9204-4cca-90fc-b1b970aa731f&items_offset={offset_no}&section_offset=3")
    desc = driver.find_elements_by_xpath('//meta[@itemprop="name"]')
    de = driver.find_elements_by_xpath('//div[@itemprop="itemListElement"]')
    asc = []
    new = []
    word = "in"
    for i in range(len(de)):
        asc.append(de[i].text)
    for i in range(len(asc)):
        new = str(asc[i]).split("\n").copy()
        finaldata=[]
        if len(str(asc[i]).split("\n")) ==11:
            if new[0] == 'HOST':
                finaldata.append("HOST")
            else:
                finaldata.append("SUPERHOST")
            finaldata.append(new[1])
            finaldata.append(new[1].partition(word)[2].replace(" ",""))
            finaldata.append(new[2])
            finaldata.append(int(str(new[3]).split(' ')[0]))
            finaldata.append(float(str(new[5]).split(" ")[1]))
            finaldata.append(int(str(new[7]).split(" ")[0]))
            finaldata.append(int(float(re.split("\$| ",new[10])[1])))
        if len(str(asc[i]).split("\n")) == 10:
            if new[0] == 'HOST':
                finaldata.append("HOST")
            else:
                finaldata.append("SUPERHOST")
            finaldata.append(new[0])
            finaldata.append(new[0].partition(word)[2].replace(" ",""))
            finaldata.append(new[1])
            finaldata.append(int(str(new[2]).split(' ')[0]))
            finaldata.append(float(str(new[4]).split(" ")[1]))
            finaldata.append(int(str(new[6]).split(" ")[0]))
            finaldata.append(int(float(re.split("\$| ",new[9])[1])))
        if len(str(asc[i]).split("\n")) == 6:
            if new[0] == 'HOST':
                finaldata.append("HOST")
            else:
                finaldata.append("SUPERHOST")
                finaldata.append(new[0])
                finaldata.append(new[0].partition(word)[2].replace(" ",""))
                finaldata.append(new[1])
                finaldata.append(int(str(new[2]).split(' ')[0]))
                finaldata.append("na")
                finaldata.append("na")
                finaldata.append(int(float(re.split("\$| ",new[5])[1])))
        temp.append(finaldata)
        data = pd.DataFrame(temp)
        
    

In [589]:
data

,0,1,2,3,4,5,6,7
0,SUPERHOST,Entire apartment in Soho,Soho,DISCOUNTED Nice beach inspired apartment in SoHo,2.0,na,na,50.0
1,SUPERHOST,Hotel room in Manhattan,Manhattan,City Queen,2.0,4.54,188,69.0
2,SUPERHOST,Private room in Manhattan,Manhattan,Charming Room in New York City,2.0,4.75,4,47.0
3,SUPERHOST,Private room in South Street Seaport,SouthStreetSeaport,Lovely Room with Great Views at Financial Dist...,2.0,4.62,13,56.0
4,SUPERHOST,Hotel room in Manhattan,Manhattan,Beautiful King Bed Hotel Room,2.0,5,6,79.0
...,...,...,...,...,...,...,...,...
295,SUPERHOST,Entire apartment in Brooklyn,Brooklyn,Modern Beautiful Bedford Stuyvesant Brownstone,4.0,4.79,203,79.0
296,SUPERHOST,Private room in Bushwick,Bushwick,Private Bedroom in the heart of Bushwick!,1.0,na,na,29.0
297,SUPERHOST,Private room in Manhattan,Manhattan,"Safe, simple, quiet and clean room near Chinatown",2.0,na,na,32.0
298,SUPERHOST,Private room in Bushwick,Bushwick,PRETTY POPULAR ON PUTNAM,2.0,4.91,88,49.0


In [590]:
data.rename(columns={0:'host_type', 1:'room_type', 2:'location', 3:'room_desc', 4:'no_of_guests',5:'ratings',6:'no_of_reviews',7:'price'},inplace = True)

In [591]:
data

,host_type,room_type,location,room_desc,no_of_guests,ratings,no_of_reviews,price
0,SUPERHOST,Entire apartment in Soho,Soho,DISCOUNTED Nice beach inspired apartment in SoHo,2.0,na,na,50.0
1,SUPERHOST,Hotel room in Manhattan,Manhattan,City Queen,2.0,4.54,188,69.0
2,SUPERHOST,Private room in Manhattan,Manhattan,Charming Room in New York City,2.0,4.75,4,47.0
3,SUPERHOST,Private room in South Street Seaport,SouthStreetSeaport,Lovely Room with Great Views at Financial Dist...,2.0,4.62,13,56.0
4,SUPERHOST,Hotel room in Manhattan,Manhattan,Beautiful King Bed Hotel Room,2.0,5,6,79.0
...,...,...,...,...,...,...,...,...
295,SUPERHOST,Entire apartment in Brooklyn,Brooklyn,Modern Beautiful Bedford Stuyvesant Brownstone,4.0,4.79,203,79.0
296,SUPERHOST,Private room in Bushwick,Bushwick,Private Bedroom in the heart of Bushwick!,1.0,na,na,29.0
297,SUPERHOST,Private room in Manhattan,Manhattan,"Safe, simple, quiet and clean room near Chinatown",2.0,na,na,32.0
298,SUPERHOST,Private room in Bushwick,Bushwick,PRETTY POPULAR ON PUTNAM,2.0,4.91,88,49.0


In [592]:
data.to_csv('airbnb.csv',index=False)